In [2]:
import pandas
from tqdm.notebook import tqdm

In [36]:
edge_df = pandas.read_csv('fixed_nucl_v5_pd_matrix_filtered_w001_aln500_len1500_edges_20250122_230906.csv', sep=',') #formerly matrix_df
idmap = pandas.read_csv('/home/mf019/longread_pangenome/expanded_dataset_analysis/output/genotyping/replicons/calls_v10/best_hits_1000bp_v10.2.csv', sep=',')

In [37]:
fixed_map = idmap[['assembly_id', 'contig_id', 'plasmid_name']]
fixed_map['contig_id'] = idmap['contig_id'].apply(lambda x: x.split('[')[0])
fixed_map['contig_id'] = fixed_map['contig_id'].apply(lambda x: x.split(' ')[0])
fixed_map['name'] = fixed_map.apply(lambda x: x['assembly_id'] + '__' + x['contig_id'], axis=1)
fixed_map = fixed_map[['name', 'plasmid_name']]
pmap = {}
for _, row in fixed_map.iterrows():
    pmap[row['name']] = row['plasmid_name']

/tmp/ipykernel_13317/824891202.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixed_map['contig_id'] = idmap['contig_id'].apply(lambda x: x.split('[')[0])
/tmp/ipykernel_13317/824891202.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixed_map['contig_id'] = fixed_map['contig_id'].apply(lambda x: x.split(' ')[0])
/tmp/ipykernel_13317/824891202.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [38]:
pmap

{'B331P__contig_1': 'chromosome',
 'B331P__contig_10': 'cp9',
 'B331P__contig_11': 'lp17',
 'B331P__contig_12': 'lp25',
 'B331P__contig_13': 'lp28-3',
 'B331P__contig_14': 'lp28-1',
 'B331P__contig_15': 'lp28-6',
 'B331P__contig_16': 'lp36',
 'B331P__contig_17': 'lp38',
 'B331P__contig_18': 'lp54',
 'B331P__contig_19': 'lp56',
 'B331P__contig_2': 'cp26',
 'B331P__contig_3': 'cp32-10',
 'B331P__contig_4': 'cp32-11',
 'B331P__contig_5': 'cp32-8',
 'B331P__contig_6': 'cp32-4',
 'B331P__contig_7': 'cp32-5',
 'B331P__contig_8': 'cp32-6',
 'B331P__contig_9': 'cp32-7',
 'B418P__contig_1': 'chromosome',
 'B418P__contig_10': 'lp17',
 'B418P__contig_11': 'lp25',
 'B418P__contig_12': 'lp28-2',
 'B418P__contig_13': 'lp28-3',
 'B418P__contig_14': 'lp28-4',
 'B418P__contig_15': 'lp28-6',
 'B418P__contig_16': 'lp36',
 'B418P__contig_17': 'lp38',
 'B418P__contig_18': 'lp54',
 'B418P__contig_2': 'cp26',
 'B418P__contig_3': 'cp32-10',
 'B418P__contig_4': 'cp32-12',
 'B418P__contig_5': 'cp32-4',
 'B418P_

In [39]:
edge_df

,Source,Target,alignment_length,weight,source_length,target_length,interaction
0,B418P__contig_1,B331P__contig_1,897953,0.994176,902773,903654,interacts
1,B331P__contig_1,B500P__contig_1,903564,0.999897,903654,903660,interacts
2,ESI26H__contig_1,B331P__contig_1,898120,0.990182,910396,903654,interacts
3,ESI361H__contig_1,B331P__contig_1,897665,0.989933,909934,903654,interacts
4,ESI403H__contig_1,B331P__contig_1,897628,0.989615,910442,903654,interacts
...,...,...,...,...,...,...,...
343334,XYZ459H__contig_2,XYZ459H__contig_5,1264,0.040237,32754,30073,interacts
343335,XYZ459H__contig_2,XYZ459H__contig_9,1082,0.035793,32754,27704,interacts
343336,XYZ459H__contig_7,XYZ459H__contig_3,792,0.026385,29476,30559,interacts
343337,XYZ459H__contig_9,XYZ459H__contig_5,17899,0.619589,27704,30073,interacts


In [40]:
df = matrix_df.copy()
min_contig_length = 750
min_weight = 0.0
source = []
target = []
weight = []
length_i = []  # Store lengths for debugging
length_j = []
align_length = []

In [41]:
edge_df['source_plasmid'] = edge_df['Source'].apply(lambda x: pmap.get(x, 'unknown'))
edge_df['target_plasmid'] = edge_df['Target'].apply(lambda x: pmap.get(x, 'unknown'))

In [56]:
# obliterate these two contigs specifically.
print(edge_df[edge_df['Target']=='URI117H__contig_26'])#ESI425H__contig_17
edge_df.drop(index=44199, inplace=True)
print(edge_df[edge_df['Target']=='URI117H__contig_26'])#ESI425H__contig_17

                   Source              Target  alignment_length    weight  \
44199  ESI425H__contig_17  URI117H__contig_26              3113  0.998076   

       source_length  target_length interaction source_plasmid target_plasmid  
44199           3119           3119   interacts     chromosome     chromosome  
Empty DataFrame
Columns: [Source, Target, alignment_length, weight, source_length, target_length, interaction, source_plasmid, target_plasmid]
Index: []


In [53]:
# this is the weird lp54 to chromosome node that doesn't exist... 
edge_df[edge_df['Source']=='GCF_040790775.1_ASM4079077v1_genomic__NZ_CP161185.1'][['source_plasmid', 'target_plasmid']]
# literally every single connection is source: cp26, target: cp26

,source_plasmid,target_plasmid
3925,cp26,cp26
20435,cp26,cp26
30501,cp26,cp26
34253,cp26,cp26
35499,cp26,cp26
42923,cp26,cp26
63184,cp26,cp26
69628,cp26,cp26
73660,cp26,cp26
87936,cp26,cp26


In [61]:
print(edge_df[edge_df['Target']=='XYZ459H__contig_10'])

                   Source              Target  alignment_length    weight  \
3988      B331P__contig_2  XYZ459H__contig_10             25920  0.977320   
12018     B418P__contig_2  XYZ459H__contig_10             26158  0.986443   
20498     B500P__contig_2  XYZ459H__contig_10             25921  0.977247   
30564    ESI26H__contig_9  XYZ459H__contig_10             25906  0.976756   
34316   ESI361H__contig_7  XYZ459H__contig_10             25856  0.974723   
...                   ...                 ...               ...       ...   
342016  URI93H__contig_16  XYZ459H__contig_10             26072  0.983200   
342443  UWI247P__contig_2  XYZ459H__contig_10             25876  0.975385   
342777  UWI248P__contig_2  XYZ459H__contig_10             25871  0.975197   
343068  UWI263P__contig_2  XYZ459H__contig_10             25880  0.975573   
343244  UWI283P__contig_2  XYZ459H__contig_10             26414  0.995478   

        source_length  target_length interaction source_plasmid target_plas

In [58]:
edge_df.to_csv('nucl_v5_matrix_edges_repl_20250122_2111.csv', index=False)